In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle

In [4]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M')
df = pd.read_csv('/content/train.csv', parse_dates=['date'], index_col='date', date_parser=dateparse)
real_test = pd.read_csv('/content/test.csv', parse_dates=['date'],index_col='date',date_parser=dateparse)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


In [6]:
data_train = []
data_test = []
num_fold = 10
length_fold = len(df)//num_fold
for i in range(num_fold):
  begin = i*length_fold
  end = begin + length_fold if i!=9 else len(df)
  data_test.append(df[begin:end])
  set_diff_df = pd.concat([df, df[begin:end], df[begin:end]]).drop_duplicates(keep=False)
  data_train.append(set_diff_df)

In [8]:
def create_features(df, label=None):
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    X = df[['hour', 'dayofweek', 'quarter', 'month', 'dayofyear', 'dayofmonth', 'weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [9]:
best_i = 0
loss = 9999999
max_train = np.zeros(num_fold, dtype = int)
min_train = np.zeros(num_fold, dtype = int)
max_test = np.zeros(num_fold, dtype = int)
min_test = np.zeros(num_fold, dtype = int)
for i in range(num_fold):
  X_train, y_train = create_features(data_train[i], label='speed')
  X_test, y_test = create_features(data_test[i], label='speed')

  reg = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05)
  
  reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=10)
  y_pred = reg.predict(X_test)

  new_loss = sum((y_pred - y_test)**2)/len(y_pred)
  print("=================================")
  print("(%d) MSE:%.8f"%(i, new_loss))
  if i==0 or new_loss < loss:
    pickle.dump(reg, open("/content/reg_best.dat", "wb"))
    loss = new_loss
    best_i = i
print(best_i)

[08:50:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:33.282	validation_1-rmse:33.6233
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:31.669	validation_1-rmse:32.0096
[2]	validation_0-rmse:30.1391	validation_1-rmse:30.4791
[3]	validation_0-rmse:28.6883	validation_1-rmse:29.0279
[4]	validation_0-rmse:27.3128	validation_1-rmse:27.652


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package s

[5]	validation_0-rmse:26.0088	validation_1-rmse:26.3479
[6]	validation_0-rmse:24.7731	validation_1-rmse:25.1121
[7]	validation_0-rmse:23.6023	validation_1-rmse:23.9411
[8]	validation_0-rmse:22.4933	validation_1-rmse:22.832
[9]	validation_0-rmse:21.4322	validation_1-rmse:21.7713
[10]	validation_0-rmse:20.4272	validation_1-rmse:20.7668
[11]	validation_0-rmse:19.4755	validation_1-rmse:19.8154
[12]	validation_0-rmse:18.5747	validation_1-rmse:18.9153
[13]	validation_0-rmse:17.7224	validation_1-rmse:18.0635
[14]	validation_0-rmse:16.9154	validation_1-rmse:17.2574
[15]	validation_0-rmse:16.1523	validation_1-rmse:16.4954
[16]	validation_0-rmse:15.4311	validation_1-rmse:15.7783
[17]	validation_0-rmse:14.7494	validation_1-rmse:15.0977
[18]	validation_0-rmse:14.1057	validation_1-rmse:14.4581
[19]	validation_0-rmse:13.4979	validation_1-rmse:13.8515
[20]	validation_0-rmse:12.9246	validation_1-rmse:13.2823
[21]	validation_0-rmse:12.384	validation_1-rmse:12.7429
[22]	validation_0-rmse:11.8749	validat

In [14]:
reg_mse_16 = pickle.load(open("/content/reg_best.dat", "rb"))
X_test, y_test = create_features(data_test[best_i], label='speed')
y_pred = reg_mse_16.predict(X_test)

[08:53:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [22]:
X_real_test = create_features(real_test)
y_pred_real = reg_mse_16.predict(X_real_test)
y_pred_real

array([49.20265 , 48.688686, 35.513554, ..., 45.514214, 39.237957,
       41.8273  ], dtype=float32)

In [24]:
id = range(len(y_pred_real))
sub = pd.DataFrame({'id':id,'speed':y_pred_real}).set_index('id')
sub.to_csv('/content/result_cv.csv')